In [1]:
from PIL import Image

import os
import urllib2
import csv
import numpy as np

In [2]:
# number of valid images
total_items = 5898

In [3]:
# dictionaries storing numeric values of labels
color_index_dict = dict()

# lists to store anomalies in data (TYPE LIST CAN LATER BE USED AS TEST DATA)
anom_image = list()
anom_color = list()

# arrays to save labels as integers as comptuers are better with numeric values
color_labels = np.ndarray((5898,))

# index and count to be assigned to each label and image
color_index = 0
count = 0

"""
Either both image and type will be saved or none of them.
"""
with open('Footwear Dataset - Sheet1.csv', 'rb') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        im_url = row['thumbnail']
        im_color = row['Color']
    
        # separating images with wrong url (RARE)
        if im_url == '' or type(im_url) != str or not im_url.startswith('http'):
            anom_image.append(count)
            continue
        
        # separating types with no or wrong labels (FREQUENT)
        if im_color == '' or type(im_color) != str:
            anom_type.append(count)
            continue

        try:
            im = Image.open(urllib2.urlopen(im_url))
        except Exception as e:
            anom_image.append(count)
            continue
           
        if im_color not in color_index_dict:
            # if label is not already present in the dictionary
            # save label to dictionary and to labels array
            color_index_dict[im_color] = color_index
            color_labels[count] = color_index
            color_index += 1
        else:
            # else extract the index of label and save to array
            ind = color_index_dict[im_color]
            color_labels[count] = ind
        
        # saving images to data folders
        path = './data/%s' % im_color
        if not os.path.exists(path):
            os.makedirs(path)
        im.save(path + '/img%d.png' % count)

        count += 1
        if count == total_items : break

In [5]:
# convert lists to numpy arrays to save them on the disk
anom_image = np.asarray(anom_image)
anom_color = np.asarray(anom_color)

### Only 5898 images were found with correct labels

In [6]:
# save all the computed arrays
np.save('bad_image_url.npy', anom_image)
np.save('bad_label.npy', anom_color)
np.save('type_index_dict.npy', color_index_dict)
np.save('final_labels.npy', color_labels)